# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **sports or athletics** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **sports or athletics**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **sports or athletics**?  For this category we are interested in sporting events or athletics broadly, please feel free to creatively interpret the category when building your research question!

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [1]:
# colour indexed map
# comparing all ncaa teams against each other, citing number of net wins
# change colour of square accordingly
# Who are U Michigans Wolvs' closest rivals, biggest threats, and easiest prey?

In [1]:
# Let's make a web scraper
!pip install requests BeautifulSoup4
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Function to get page from its url
# Got this from https://realpython.com/python-web-scraping-practical-introduction/
def simple_get(url):
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    
def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

def log_error(e):
    print(e)

In [ ]:
# Testing new function
# Should output some big number
raw_html = simple_get('https://google.com')
len(raw_html)

In [ ]:
# This function uses BeautifulSoup to scrape data from
# pages containing Michigan's results
def get_data(url):
    '''
    Get the seasons data for Michigan
    '''
    # Dict to store results (change this to pd later)
    df = pd.DataFrame(columns=['Opponent', 'Net Wins'])
    # List of teams (for checking purposes)
    tms = []
    # Loop through seasons 2010 to 2019
    for n in range(10, 20):
        # Fetch results
        response = simple_get(url.format(n))
        # Parse with BS
        if response is not None:
            html = BeautifulSoup(response, 'html.parser')
            # For each table entry, find ones related to Big Ten Conference
            for tr in html.select('tr'):
                bigtens = tr.find('a', {'title' : 'Big Ten Conference'})
                team = ''
                res = ''
                if bigtens != None:
                    # If it's a Big Ten match, get the team name and result
                    team = tr.find('td', {'data-stat' : 'opp_name'}).find('a').contents[0]
                    res = tr.find('td', {'data-stat' : 'game_result'}).contents[0]
                    # If the team isn't already in the dict, set its value to 0
                    if team not in tms:
                        tms.append(team)
                        df = df.append(pd.DataFrame([[team, 0]], columns=['Opponent', 'Net Wins']), ignore_index=True)
                    # If result is 'W', add 1 to team's value in dictionary
                    # Otherwise, take 1 from team's value
                    if res == 'W':
                        df.loc[df['Opponent'] == team, 'Net Wins'] += 1
                    else:
                        df.loc[df['Opponent'] == team, 'Net Wins'] -= 1
                else:
                    continue
    return(df)

In [ ]:
# Use the webscraper to collate data from 2010 to 2019
src = 'https://www.sports-reference.com/cbb/schools/michigan/20{}-schedule.html'
data = get_data(src)

In [ ]:
# Table listing Wolverines' performance against other teams
data = data.sort_values(by='Net Wins', ascending=False)
data['Xpos'] = 1
data.insert(3,'Ypos', range(1, 1 + len(data)))

In [ ]:
data

In [ ]:
res = data.pivot(index='Ypos', columns='Xpos', values='Net Wins')
res

In [ ]:
names = data.pivot(index='Ypos', columns='Xpos', values='Opponent')
teams = list(names[1])
teams = [str(x) for x in teams]
tser = pd.Series(teams)
# Use this to check the data type of objects (sometimes they are NavigableStrings... bad)
# type(names.iloc[0][1])
teams

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(3,8))
title = "Michigan Wolverine's Rivals"
plt.title(title, fontsize=15)
sns.heatmap(res)
ax.axis('off')
ax.set_xticks([])
ax.set_yticks([])
plt.show()